In [1]:
import pandas as pd
import os

c:\Users\bahawalkhan\Anaconda3\envs\yolo\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\bahawalkhan\Anaconda3\envs\yolo\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\bahawalkhan\Anaconda3\envs\yolo\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
class DataGenerator(tf.keras.utils.Sequence):

    def __init__(self, df, batch_size=32,shuffle=True,image_dir=""):
        self.batch_size = batch_size
        self.df = df
        self.image_dir = image_dir
        self.indices = self.df.index.tolist()
        self.shuffle = shuffle
        self.on_epoch_end()


    def __len__(self):
        return len(self.indices) // self.batch_size

    def __getitem__(self, index):
        index = self.index[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.indices[k] for k in index]
        X, y = self.__get_data(batch)
        return X, y

    def on_epoch_end(self):
        self.index = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.index)

    def get_embedding_path(self, img_path):
        dirs = img_path.split('/')
        dirs[1] = dirs[1]+'_embeddings'
        dirs[-1] = dirs[-1].replace('jpg','npy')
        dir_path = '/'.join(dirs).lstrip('/')
        return dir_path
    

    def load_embedding(self, slice):
        img1,img2, label = self.df.iloc[slice]['data']
        os.path.basename(img1)
        img1_embeding_path = self.get_embedding_path(img1)
        img2_embeding_path = self.get_embedding_path(img2)

        embedding1 =  np.load(img1_embeding_path, allow_pickle=True)
        embedding2 = np.load(img2_embeding_path, allow_pickle=True)
        embedding = np.concatenate((embedding1, embedding2), axis=0)
        return embedding, int(label)

    def __get_data(self, batch):
        X = []
        y = []
        for i, id in enumerate(batch):
            img, label = self.load_embedding(batch[i])
       
            X.append(img)
            y.append(label)
        return np.array(X),np.array(y)

In [3]:
from random import shuffle 

def read_data_file(file_name):
    data = []
    with open(file_name, 'r') as file:
        for line in file:
            # Split each line into pairs using a comma as the separator
            pair = line.strip().split(',')
            # Append the pair to the list
            data.append(pair)
    return data

# Provide the name of the file you want to read
file_name = 'train-same_pairs.txt'
file_name2 = 'train-diff_pairs.txt'
file_name_color_swap = 'swap_test-same_pairs.txt'
file_name2_color_swap = 'swap_test-diff_pairs.txt'
# Call the function to read the data from the file
data = read_data_file(file_name)
data.extend(read_data_file(file_name2))
data.extend(read_data_file(file_name_color_swap))
data.extend(read_data_file(file_name2_color_swap))

shuffle(data)
val_file_name = 'test-2-diff_pairs.txt'
val_file_name2 = 'test-2-same_pairs.txt'
# Call the function to read the data from the file
val_data = read_data_file(file_name)
val_data.extend(read_data_file(file_name2))
shuffle(val_data)


In [61]:
balanced_data = pd.DataFrame({'data':data})

In [4]:
val_data_frame = pd.DataFrame({"data":val_data})

In [17]:
from sklearn.model_selection import train_test_split
val_df, test_df = train_test_split(val_data_frame, test_size=0.2, random_state=42)


In [64]:
# import pandas as pd
# from sklearn.model_selection import train_test_split

# # Load your dataframe

# # Split into train and test sets
# train_df, test_df = train_test_split(balanced_data, test_size=0.2, random_state=42)

# # Split train set into train and validation sets
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)


In [18]:
# partial_validation, test = train_test_split(train_df, test_size=0.2, random_state=42)
test_df.reset_index(drop=True, inplace=True)
# test.reset_index(drop=True, inplace=True)


In [66]:
# print(train_df.shape)
# train_df.head()

In [67]:
test_df.reset_index(drop=True, inplace=True)

test_df.head()

,data
0,"[../balanced_data/0321\006841.jpg, ../balanced..."
1,"[../balanced_data/0174\001262.jpg, ../balanced..."
2,[../balanced_data/red_f1\75ca545e-050e-11ee-aa...
3,"[../balanced_data/0004\005083.jpg, ../balanced..."
4,"[../balanced_data/car3\IMG_4034.MOV_15.jpg, ....."


In [19]:
# train_loader = DataGenerator(balanced_data, batch_size=256)
val_loader = DataGenerator(test_df, batch_size=256)

In [69]:
for i in val_loader:
    i
    break

In [70]:
for i,l in val_loader:
    print(i.shape)
    break

(256, 1536)


In [71]:
from tqdm import tqdm

In [72]:
# from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [5]:
import tensorflow as tf

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D,Dropout
from numpy import unique

In [7]:
from keras import backend as K

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

model = Sequential()

# Add a Conv1D layer with 32 filters and a kernel size of 3.
model.add(Conv1D(128, kernel_size=10, activation='relu',input_shape=(1536,1)))

# Add a MaxPooling1D layer with a pool size of 2.
model.add(MaxPooling1D(pool_size=2))

# Add a Conv1D layer with 64 filters and a kernel size of 3.
model.add(Conv1D(64, kernel_size=5, activation='relu'))

# Add a MaxPooling1D layer with a pool size of 2.
model.add(MaxPooling1D(pool_size=2))

# Flatten the output of the MaxPooling1D layer.
model.add(Flatten())

# Add a dropout layer with a rate of 0.2.
model.add(Dropout(0.2))

# Add a Dense layer with 128 units and a relu activation function.
model.add(Dense(128, activation='relu'))

# Add a dropout layer with a rate of 0.2.
model.add(Dropout(0.2))

# Add a Dense layer with 1 unit and a sigmoid activation function for binary classification.
model.add(Dense(1, activation='sigmoid'))

# Compile the model.
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',precision_m])


In [9]:
# model.save('last_latest')
model.load_weights('complex-conv1d-kernel-10-5.h5')

In [20]:
model.evaluate(val_loader)

3227/4691 [===================>..........] - ETA: 8:23 - loss: 0.0016 - accuracy: 0.9996 - precision_m: 0.9994

ResourceExhaustedError: Graph execution error:

2 root error(s) found.
  (0) RESOURCE_EXHAUSTED:  MemoryError: Unable to allocate 1.50 MiB for an array with shape (256, 1536) and data type float32
Traceback (most recent call last):

  File "c:\Users\bahawalkhan\Anaconda3\envs\yolo\lib\site-packages\tensorflow\python\ops\script_ops.py", line 271, in __call__
    ret = func(*args)

  File "c:\Users\bahawalkhan\Anaconda3\envs\yolo\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "c:\Users\bahawalkhan\Anaconda3\envs\yolo\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 1035, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "c:\Users\bahawalkhan\Anaconda3\envs\yolo\lib\site-packages\keras\engine\data_adapter.py", line 903, in wrapped_generator
    for data in generator_fn():

  File "c:\Users\bahawalkhan\Anaconda3\envs\yolo\lib\site-packages\keras\engine\data_a

In [42]:
""" run these command on yolo venv to convert model to trt
python -m tf2onnx.convert --saved-model last_latest --output latest.onnx
python -m onnxruntime.tools.make_dynamic_shape_fixed --dim_param unk__55 --dim_value 20 latest.onnx model.fixed.onnx
trtexec --onnx=model.fixed.onnx --saveEngine=conv1d_engine_single.trt
"""

' run these command on yolo venv to convert model to trt\npython -m tf2onnx.convert --saved-model last_latest --output latest.onnx\npython -m onnxruntime.tools.make_dynamic_shape_fixed --dim_param unk__55 --dim_value 20 latest.onnx model.fixed.onnx\ntrtexec --onnx=model.fixed.onnx --saveEngine=conv1d_engine_single.trt\n'

In [78]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
mcp = tf.keras.callbacks.ModelCheckpoint(
    filepath='data_aug_model.h5',
    save_weights_only=True,
    monitor='val_precision_m',
    mode='max',
    save_best_only=True)
import datetime
log_dir = f"logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [79]:
for i,l in train_loader:
    print(l)
    break

[1 1 0 1 0 1 0 0 1 0 0 1 1 1 1 1 1 0 0 1 1 0 1 0 0 1 0 1 0 0 0 1 0 1 0 0 1
 1 0 0 0 1 1 1 1 0 1 1 0 1 0 1 1 1 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 1 1 1
 0 0 0 1 1 1 1 0 0 1 0 0 1 0 1 1 1 1 0 0 0 0 1 0 1 1 0 1 1 0 0 0 1 1 1 1 0
 1 0 0 0 0 1 0 1 0 0 1 0 0 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 1
 0 1 0 1 0 1 1 1 0 1 0 1 1 0 0 1 1 0 0 1 1 1 0 1 1 0 1 0 1 1 0 0 1 1 1 1 1
 1 0 0 1 0 1 0 1 1 1 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 1 0 1
 1 0 0 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 1 1 0 0 1 1 1 0 1 1 0 0 1 0 0 1]


In [80]:
from tensorflow.keras.callbacks import Callback

class CustomModelCheckpoint(Callback):
    def __init__(self, filepath, save_steps):
        super(CustomModelCheckpoint, self).__init__()
        self.filepath = filepath
        self.save_steps = save_steps

    def on_epoch_end(self, epoch, logs=None):
        # Do nothing at the end of each epoch to prevent saving checkpoints based on epochs
        pass

    def on_batch_end(self, batch, logs=None):
        if (batch + 1) % self.save_steps == 0:
            self.model.save(self.filepath.format(batch + 1))


In [81]:
checkpoint_callback = CustomModelCheckpoint(filepath="save_weights/complex-model_checkpoint_kernal_size_10.h5", save_steps=20000)


In [82]:
# tf.test.is_gpu_available()
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [83]:
model.fit(train_loader, validation_data=val_loader, epochs = 20, callbacks=[mcp,tensorboard_callback,es,checkpoint_callback])

Epoch 1/20
23592/23592 [==============================] - 5041s 213ms/step - loss: 0.0101 - accuracy: 0.9965 - precision_m: 0.9958 - val_loss: 0.0017 - val_accuracy: 0.9995 - val_precision_m: 0.9992
Epoch 2/20
  977/23592 [>.............................] - ETA: 55:15 - loss: 0.0085 - accuracy: 0.9970 - precision_m: 0.9963

KeyboardInterrupt: 

In [28]:
model.save('last.h5')

In [ ]:
model.evaluate(val_loader)